
## Introducción a los componentes fundamentales de Langmem: `MemoryStoreManager` y `MemoryManager`

En el ecosistema de **Langmem**, la gestión de la memoria es un aspecto crucial para el funcionamiento eficiente y coherente de agentes inteligentes y sistemas conversacionales. Dos bloques fundamentales en esta arquitectura son:

- **MemoryStoreManager**: Responsable de gestionar el almacenamiento físico de los recuerdos o memorias. Administra las distintas fuentes de almacenamiento, ya sea en memoria local, bases de datos, o sistemas de almacenamiento distribuidos.

- **MemoryManager**: Actúa como el orquestador lógico de las memorias. Controla cómo se crean, actualizan, recuperan y eliminan las memorias en función del contexto y las necesidades del agente.

En esta sección, exploraremos en detalle el rol y la interacción de estos dos componentes, entendiendo cómo permiten a Langmem construir agentes más adaptativos, contextuales y persistentes en su conocimiento.


In [8]:
# Importamos las clases y tipos necesarios de langgraph
from langgraph.store.base import BaseStore, Item, Op, Result
from langgraph.store.memory import InMemoryStore
from typing import Any, Iterable, Literal, NamedTuple, Optional, Union, cast
from langchain_aws import ChatBedrockConverse
import boto3
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.config import RunnableConfig
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langgraph.func import entrypoint
from langgraph.store.memory import InMemoryStore
from langmem import create_memory_store_manager
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain_aws import ChatBedrockConverse
from langchain_aws import ChatBedrock
from IPython.display import Image, display
from langmem import create_memory_manager # 
from pydantic import BaseModel
# Definimos una clase personalizada que extiende de BaseStore
class CustomMemoryStore(BaseStore):
    """
    CustomMemoryStore es una implementación personalizada de BaseStore que
    actúa como adaptador para un almacenamiento externo (ext_store).
    Permite operaciones de get, put y batch de forma síncrona y asíncrona.
    """

    def __init__(self, ext_store):
        """
        Inicializa el CustomMemoryStore con un almacén externo.

        Args:
            ext_store: Un objeto que implementa los métodos get, put y batch.
        """
        self.store = ext_store

    def get(self, namespace: tuple[str, ...], key: str) -> Optional[Item]:
        """
        Recupera un ítem del almacén dado un namespace y una clave.

        Args:
            namespace (tuple): Un espacio de nombres como tupla de strings.
            key (str): La clave del ítem a recuperar.

        Returns:
            Optional[Item]: El ítem encontrado o None si no existe.
        """
        return self.store.get(namespace, key)

    def put(self, namespace: tuple[str, ...], key: str, value: dict[str, Any]) -> None:
        """
        Guarda un ítem en el almacén bajo un namespace y una clave.

        Args:
            namespace (tuple): El espacio de nombres donde guardar.
            key (str): La clave bajo la cual guardar el ítem.
            value (dict): El valor del ítem a guardar.
        """
        print(f"PUT::namespace={namespace}, key={key}, value={value}:")
        return self.store.put(namespace, key, value)

    async def aget(self, namespace: tuple[str, ...], key: str) -> Optional[Item]:
        """
        Recupera un ítem de manera asíncrona.

        Nota: Aunque se define como async, internamente llama al método síncrono get.

        Args:
            namespace (tuple): Espacio de nombres.
            key (str): Clave a recuperar.

        Returns:
            Optional[Item]: El ítem encontrado o None.
        """
        res = await self.get(namespace, key)  # Puede lanzar advertencias ya que get no es async
        return res

    async def aput(self, namespace: tuple[str, ...], key: str, value: dict[str, Any]) -> None:
        """
        Guarda un ítem de forma asíncrona.

        Nota: Llama al método síncrono put ya que no existe una versión nativa async.

        Args:
            namespace (tuple): Espacio de nombres.
            key (str): Clave donde guardar.
            value (dict): El valor a guardar.
        """
        res = self.put(namespace, key, value)  # No se puede await porque put es síncrono
        print(f"ASYN-PUT::::namespace={namespace}, key={key}, value={value}:")
        return None

    def batch(self, ops: Iterable[Op]) -> list[Result]:
        """
        Ejecuta una serie de operaciones en lote de forma síncrona.

        Args:
            ops (Iterable[Op]): Lista de operaciones a realizar.

        Returns:
            list[Result]: Resultados de las operaciones.
        """
        return self.store.batch(ops)

    async def abatch(self, ops: Iterable[Op]) -> list[Result]:
        """
        Ejecuta una serie de operaciones en lote de manera asíncrona.

        Args:
            ops (Iterable[Op]): Lista de operaciones a realizar.

        Returns:
            list[Result]: Resultados de las operaciones.
        """
        print(f"ASYNC::BATCH::::ops={ops}:")
        res = await self.store.abatch(ops)
        return res


AttributeError: module 'typing' has no attribute 'NotRequired'

In [7]:
%pip install langmem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


- Buena práctica: Podrías considerar envolver self.get en aget usando asyncio.to_thread para evitar advertencias de tipo (aunque depende del rendimiento que quieras optimizar).

In [5]:

class Triple(BaseModel): # 
    """Store all new facts, preferences, and relationships as triples."""
    subject: str
    predicate: str
    object: str
    context: str | None = None

# Set up storage 
in_memory_store = CustomMemoryStore(InMemoryStore())
# ---- ⚠️ Actualiza la región para tu configuración de AWS ⚠️ ----
region = 'us-east-2'

bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

bedrock_client = boto3.client("bedrock-runtime", region_name=region)

model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

llm = ChatBedrockConverse(
    model=model_id,
    temperature=0,
    max_tokens=5000,
    client=bedrock_client,
)

NameError: name 'BaseModel' is not defined